In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import db_builder.db_handler as dbh
import logging

In [ ]:
logging.getLogger('sqlalchemy').setLevel(logging.WARNING)

In [ ]:
db_params = {
    'user': 'postgres',
    'password': input('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'ttdatabase', #tensionTerminator
}

In [ ]:
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
engine  = toolcheck.get_engine()

In [ ]:
def table_to_df(tabel_name: str):
    return pd.read_sql_table(tabel_name, con=engine)

In [ ]:
loop_df = table_to_df('loop')

table_name = 'loop'  # Replace 'your_table_name' with the actual name of the table
df = pd.read_sql_table(table_name, con=engine)

In [ ]:
loop_df

In [ ]:
def overview():
    print(f"Die DB enthält {loop_df['timestamp'].count()} Aufzeichnungen, das entspricht {loop_df['session_length'].sum()} Sekunden an Videomaterial:")
    print(f"\t{loop_df['human_labeled'].sum()} Aufzeichnungen sind von Menschen gelabelt ({loop_df.loc[loop_df['human_labeled'] == True, 'session_length'].sum()} Sekunden) ")
    print(f"\t{(loop_df['human_labeled'] == False).sum()} Aufzeichnungen sind von Menschen gelabelt ({loop_df.loc[loop_df['human_labeled'] == False, 'session_length'].sum()} Sekunden)")
    print(f"Videolänge mean: {loop_df['session_length'].mean()} Sekunden")
    print(f"Videolänge median: {loop_df['session_length'].median()} Sekunden")
    print(f"Videolänge min: {loop_df['session_length'].min()} Sekunden")
    print(f"Videolänge max: {loop_df['session_length'].max()} Sekunden")

In [ ]:
overview()

In [ ]:
grouped_df = loop_df.groupby(loop_df['timestamp'].dt.date)['session_length'].sum().reset_index()

In [ ]:
grouped_df['labeled'] = loop_df[loop_df['human_labeled'] == True].groupby(loop_df['timestamp'].dt.date)['session_length'].sum().reset_index()['session_length']



In [ ]:
grouped_df = grouped_df.fillna(0)

In [ ]:
grouped_df

In [ ]:
loop_df.loc[loop_df['human_labeled'] == True, 'session_length'].sum()

In [ ]:
grouped_df['session_length']/60

In [ ]:
label = ['label', 'no lable']
plt.figure(figsize=(12, 10))

plt.bar(grouped_df['timestamp'].astype(str), grouped_df['session_length']/60, label='no label')
plt.bar(grouped_df['timestamp'].astype(str), grouped_df['labeled']/60, label='label')
plt.xlabel('Datum')
plt.ylabel('Videomaterial in Minuten')
plt.title('Aufgezeichnete Daten pro Tag')
plt.ylim(0,70)
plt.legend()

plt.show()



In [ ]:
loop_df

In [ ]:
session_user_id_df_test = loop_df.groupby(loop_df['timestamp'].dt.date)['user_id'].unique()

In [ ]:
session_user_id_df_test

In [ ]:
session_user_id_df_test1 = loop_df.groupby([loop_df['timestamp'].dt.date, 'user_id']).size().unstack(fill_value=0)

In [ ]:
session_user_id_df_test1

In [ ]:
session_user_time_df = loop_df.groupby([loop_df['timestamp'].dt.date, 'user_id'])['session_length'].sum().unstack(fill_value=0)

In [ ]:
session_user_time_df = session_user_time_df.reset_index()

In [ ]:
session_user_time_df

In [ ]:
users_df = table_to_df('users')

In [ ]:
def get_user(id):
    user_df = table_to_df('users')
    users_loc = user_df.loc[user_df['id']==id].values
    return f"{users_loc[0][1]} {users_loc[0][2]}"

In [ ]:
#label = ['label', 'no lable']
plt.figure(figsize=(12, 10))
bottom = np.zeros(3)

for x in range(loop_df['user_id'].nunique()):
    plt.bar(session_user_time_df['timestamp'].astype(str), session_user_time_df[x+1]/60, label=f"{get_user(x+1)}", bottom=bottom)
    bottom += session_user_time_df[x+1]/60
#plt.bar(grouped_df['timestamp'].astype(str), grouped_df['labeled']/60, label='label')
plt.xlabel('Datum')
plt.ylabel('Videomaterial in Minuten')
plt.ylim(0,70)
plt.title('Aufgezeichnete Daten pro Tag, pro Person')
plt.legend()

plt.show()